In [1]:
import geotransfer
from geotransfer.label import *
from geotransfer.sen12ms_dataLoader import *
from geotransfer.preprocessing import *
from geotransfer.model_pre_functions import *
from geotransfer.cca_core import *
from geotransfer.get_activations import *

In [2]:
import yaml
import pandas as pd
from tqdm import tqdm
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

In [7]:
conf_dict = [{'data_dir':"/Volumes/ksankaran/sen12ms_subsample2/smaller_subset/metadata_clusters.csv",
        'output_dir':'/Volumes/ksankaran/sen12ms_subsample2/smaller_subset/'},
        {'imagenet':False,'model_size':'tiny','num_freeze':5},
       {'source_cluster':1,'test_size':0.3},
       {'n_epochs':2,'batch_size':8}]
with open('/Volumes/ksankaran/sen12ms_subsample2/smaller_subset/conf.yaml', 'w') as f:
    data = yaml.dump(conf_dict, f)

## Read parameters from a configuration file.

In [8]:
conf = yaml.load(open("/Volumes/ksankaran/sen12ms_subsample2/smaller_subset/conf.yaml"))

/Users/miaoxinran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


In [10]:
metadata_dir = conf[0]['data_dir']
output_dir = conf[0]['output_dir']

imagenet = conf[1]['imagenet']
model_size = conf[1]['model_size']
num_freeze = conf[1]['num_freeze']

source_cluster = conf[2]['source_cluster']
test_size = conf[2]['test_size']

batch_size = conf[3]['batch_size']
n_epochs = conf[3]['n_epochs']


In [24]:
metadata = pd.read_csv(metadata_dir)
metadata.head()

,Unnamed: 0,Unnamed: 0.1,image_basename,label_basename,roi,season,scene,patch,image_path,label_path,longitude,latitude,id,label_id,clusters
0,178,179,ROIs1970_fall_s2_130_p503.tif,ROIs1970_fall_lc_130_p503.tif,ROIs1970,fall,130,p503,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/s2_13...,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/lc_13...,-76.680057,39.189120,190,1,0
1,260,261,ROIs1970_fall_s2_130_p839.tif,ROIs1970_fall_lc_130_p839.tif,ROIs1970,fall,130,p839,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/s2_13...,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/lc_13...,-76.680057,39.189120,190,1,0
2,605,606,ROIs1970_fall_s2_114_p921.tif,ROIs1970_fall_lc_114_p921.tif,ROIs1970,fall,114,p921,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/s2_11...,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/lc_11...,-83.955012,35.775723,179,1,0
3,138,139,ROIs1970_fall_s2_21_p320.tif,ROIs1970_fall_lc_21_p320.tif,ROIs1970,fall,21,p320,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/s2_21...,/Volumes/ksankaran/Sen12MS/ROIs1970_fall/lc_21...,-2.083650,35.050345,131,2,1
4,487,488,ROIs2017_winter_s2_130_p70.tif,ROIs2017_winter_lc_130_p70.tif,ROIs2017,winter,130,p70,/Volumes/ksankaran/Sen12MS/ROIs2017_winter/s2_...,/Volumes/ksankaran/Sen12MS/ROIs2017_winter/lc_...,48.558600,31.321705,246,2,1


## Load data and split into source, target (train, test) subsets

In [6]:
X,y = load_data.load_img(metadata)

100%|██████████| 27/27 [00:31<00:00,  1.16s/it]


In [30]:
y = y-1

In [9]:
#np.save('/Volumes/ksankaran/sen12ms_subsample2/smaller_subset/X',X)
#np.save('/Volumes/ksankaran/sen12ms_subsample2/smaller_subset/y',y)
#X = np.load('/Volumes/ksankaran/sen12ms_subsample2/smaller_subset/X.npy')
#y = np.load('/Volumes/ksankaran/sen12ms_subsample2/smaller_subset/y.npy')

In [31]:
X_source, y_source, X_target_train, X_target_test,  y_target_train, y_target_test, X_target,y_target = split_data(metadata,
                                                                                                                  X,y,
                                                                                                                  source_cluster = source_cluster,
                                                                                                                  test_size = test_size)

In [32]:
X_source.shape,y_source.shape,X_target_train.shape,y_target_train.shape,X_target_test.shape,y_target_test.shape

((15, 13, 256, 256), (15,), (8, 13, 256, 256), (8,), (4, 13, 256, 256), (4,))

In [33]:
d2 = get_dict(y_source,num_classes=10)

In [36]:
train_x,train_y = prepare_data(X_source,y_source,d2)
train_x.shape,train_y.shape

(torch.Size([15, 13, 256, 256]), torch.Size([15]))

In [20]:
val_x,val_y = prepare_data(X_target_train,y_target_train,d2)
val_x.shape,val_y.shape

(torch.Size([8, 13, 256, 256]), torch.Size([8]))

In [39]:
test_x,test_y = prepare_data(X_target_test,y_target_test,d2)
test_x.shape,test_y.shape

(torch.Size([4, 13, 256, 256]), torch.Size([4]))

## Train the model

In [16]:
if imagenet == True:
    model = torch.load('/Volumes/ksankaran/ImageNet/Models/sub_mod1')
else if model_size == 'tiny':
    model = Net()
else if model_size =='small':
    model = smallNet()

In [17]:
optimizer = Adam(model.parameters(), lr=0.0001)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [16]:
model = model.double()

In [17]:
train_model(train_x,train_y, model, criterion, optimizer,
                n_epochs=n_epochs,
                batch_size=batch_size)

 50%|█████     | 1/2 [00:52<00:52, 52.51s/it]

epoch: 	 1 	 training loss: 	 2.0495978062270326


100%|██████████| 2/2 [01:45<00:00, 52.74s/it]

epoch: 	 2 	 training loss: 	 0.678595673071245


Net(
  (cnn_layers): Sequential(
    (0): Conv2d(13, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=1, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (13): BatchNorm2d(512, eps=1e-

In [18]:
pred_acc(train_x,train_y,model)

100%|██████████| 2/2 [00:14<00:00,  7.25s/it]


0.8035714285714286

In [21]:
pred_acc(val_x,val_y,model,batch_size=8)

100%|██████████| 1/1 [00:07<00:00,  7.16s/it]


0.0

In [117]:
for param in model.cnn_layers[0:num_freeze].parameters():
    param.requires_grad = False
for param in model.cnn_layers[(num_freeze+1):].parameters():
    param.requires_grad = True
for parameter in model.linear_layers.parameters():
    parameter.requires_grad = True

In [36]:
train_model(val_x,val_y, model, criterion, optimizer,
                n_epochs=2,
                batch_size=8)

 50%|█████     | 1/2 [00:12<00:12, 12.35s/it]

epoch: 	 1 	 training loss: 	 2.9449863463317887


100%|██████████| 2/2 [00:27<00:00, 13.76s/it]

epoch: 	 2 	 training loss: 	 2.4436558488803066


Net(
  (cnn_layers): Sequential(
    (0): Conv2d(13, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=1, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (12): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (13): BatchNorm2d(512, eps=1e-

In [37]:
#torch.save(model,'/Volumes/ksankaran/sen12ms_subsample2/smaller_subset/model_cbr_trained.pt')

In [38]:
pred_acc(val_x,val_y,model,batch_size=batch_size)

100%|██████████| 1/1 [00:06<00:00,  6.86s/it]


0.375

In [40]:
pred_acc(test_x,test_y,model,batch_size=batch_size)

100%|██████████| 1/1 [00:03<00:00,  3.55s/it]


0.0

## Load and save the activations.

In [133]:
test_acts = loader_activations(DataLoader(TensorDataset(test_x,test_y)),model,model.cnn_layers)
val_acts = loader_activations(DataLoader(TensorDataset(val_x,val_y)),model,model.cnn_layers)
train_acts = loader_activations(DataLoader(TensorDataset(train_x,train_y)),model,model.cnn_layers)

In [153]:
torch.save(test_acts,output_dir+'test_acts.pth')
torch.save(val_acts,output_dir+'val_acts.pth')
torch.save(val_acts,output_dir+'val_acts.pth')

In [ ]:
torch.save(model,output_dir+"model.pt")